In [74]:
import requests
from bs4 import BeautifulSoup

In [36]:
isa_base_url = 'http://isa.epfl.ch/imoniteur_ISAP/'

def isa_url(path):
    return isa_base_url + path
    
isa_home_url = isa_url('%21gedpublicreports.htm?ww_i_reportmodel=133685247')

r = requests.get(isa_home_url)

In [72]:
r.status_code

200

In [73]:
home_html = r.text

In [75]:
home_soup = BeautifulSoup(home_html, 'html.parser')

In [76]:
print(home_soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <title>
    IS-Academia
   </title>
   <link href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="!ewxslsource.css?ww_x_path=Eqnx.Ajax.PageNL_css" rel="stylesheet" type="text/css">
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.core" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.drag" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.event" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.dom" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.ewnet.namespace" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.ewnet.core" type="text/javascript">
     </script>
     <script src="gedpubl

In [79]:
toc_frame = soup.find('frame', attrs={'name': 'toc'})

In [80]:
toc_frame_url = toc_frame.attrs['src']

In [81]:
toc_url = isa_url(toc_frame_url)

In [82]:
toc_url

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247'

In [83]:
toc_html = requests.get(toc_url).text

In [84]:
toc_soup = BeautifulSoup(toc_html, 'html.parser')

In [85]:
print(toc_soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <div>
   </div>
   <title>
   </title>
   <script src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr" type="text/javascript">
   </script>
   <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator" rel="stylesheet" type="text/css"/>
   </link>
  </meta>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <div class="filtres">
   <form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
    <input name="ww_b_list" type="hidden" value="1">
     <input name="ww_i_reportmodel" type="hidden" value="133685247">
      <input name="ww_c_langue" type="hidden" value="">
       <h1 id="titre">
        Liste des étudiants inscrits par semestre
       </h1>
       <table border="0" id="format">
 

In [63]:
form_elem = toc_soup.find('form', attrs={'name': 'f'})

In [66]:
print(form_elem.prettify())

<form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
 <input name="ww_b_list" type="hidden" value="1">
  <input name="ww_i_reportmodel" type="hidden" value="133685247">
   <input name="ww_c_langue" type="hidden" value="">
    <h1 id="titre">
     Liste des étudiants inscrits par semestre
    </h1>
    <table border="0" id="format">
     <tr>
      <th>
       Format:
      </th>
     </tr>
     <tr>
      <td>
       <input checked="" name="ww_i_reportModelXsl" type="radio" value="133685270">
        html
       </input>
      </td>
     </tr>
     <tr>
      <td>
       <input name="ww_i_reportModelXsl" type="radio" value="133685271">
        xls
       </input>
      </td>
     </tr>
    </table>
    <h1>
    </h1>
    <table border="0" id="filtre">
     <tr>
      <th>
       Unité académique
      </th>
      <td>
       <input name="zz_x_UNITE_ACAD" type="hidden" value="">
        <select name="ww_x_UNITE_ACAD" onchange="document.f.zz_x_UNITE_ACAD.value=document.f.ww_x_U

In [94]:
form_action = form_elem['action']
form_action_url = isa_url(form_action)
form_action_url

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter'

In [100]:
payload = {
    'ww_b_list': '1',
    'ww_i_reportmodel': '133685247',
    'ww_c_langue': '',
    
    'ww_i_reportModelXsl': '133685270', # Format (html)
    'zz_x_UNITE_ACAD': 'Informatique',
    'ww_x_UNITE_ACAD': '249847',
    'zz_x_PERIODE_ACAD': '2016-2017',
    'ww_x_PERIODE_ACAD': '355925344',
    'zz_x_PERIODE_PEDAGO': 'Bachelor semestre 1',
    'ww_x_PERIODE_PEDAGO': '249108',
    'zz_x_HIVERETE': '',
    'ww_x_HIVERETE': 'null',
    'dummy': 'ok'
}
payload

{'dummy': 'ok',
 'ww_b_list': '1',
 'ww_c_langue': '',
 'ww_i_reportModelXsl': '133685270',
 'ww_i_reportmodel': '133685247',
 'ww_x_HIVERETE': 'null',
 'ww_x_PERIODE_ACAD': '355925344',
 'ww_x_PERIODE_PEDAGO': '249108',
 'ww_x_UNITE_ACAD': '249847',
 'zz_x_HIVERETE': '',
 'zz_x_PERIODE_ACAD': '2016-2017',
 'zz_x_PERIODE_PEDAGO': 'Bachelor semestre 1',
 'zz_x_UNITE_ACAD': 'Informatique'}

In [101]:
form_result = requests.get(form_action_url, params=payload)

In [102]:
form_result.status_code

200

In [104]:
form_result.text

'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><div></div><title></title><script type="text/javascript" src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr"></script><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style"><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><div class="filtres"><form method="GET" name="f" action="!GEDPUBLICREPORTS.filter"><input type="hidden" name="ww_b_list" value="1"><input type="hidden" name="ww_i_reportmodel" value="133685247"><input type="hidden" name="ww_c_langue" value=""><h1 id="titre">Liste des étudiants inscrits par semestre</h1><table border="0" id="format"><tr><th>Format:</th></tr><tr><td><input type="radio" name="ww_i_reportModelXsl" value="133685270" checked>html</td></t

In [105]:
form_result_soup = BeautifulSoup(form_result.text, 'html.parser')

In [106]:
print(form_result_soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <div>
   </div>
   <title>
   </title>
   <script src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr" type="text/javascript">
   </script>
   <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator" rel="stylesheet" type="text/css"/>
   </link>
  </meta>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <div class="filtres">
   <form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
    <input name="ww_b_list" type="hidden" value="1">
     <input name="ww_i_reportmodel" type="hidden" value="133685247">
      <input name="ww_c_langue" type="hidden" value="">
       <h1 id="titre">
        Liste des étudiants inscrits par semestre
       </h1>
       <table border="0" id="format">
 

In [179]:
import re

# loadReport('ww_x_GPS=2021043255');return false;

parse_url_pattern = r"""'(\w+)=(\d+)'"""
parse_url_re = re.compile(parse_url_pattern)

def parse_url(url):
    print(url)
    match = re.search(parse_url_pattern, url)
    print(match)
    if match == None:
        return None
    
    return {
        'key': match.group(1),
        'value': match.group(2)
    }
    
# Informatique, 2016-2017, Bachelor semestre 1

parse_item_pattern = r"""([\w\s]+),\s*(\d+)-(\d+),\s*([\w\s]+)"""
parse_item_re = re.compile(parse_item_pattern)

def parse_item(item, url, payload):
    match = re.match(parse_item_re, item)
    if match == None:
        return None
    
    parsed_url = parse_url(url)
    
    params = payload.copy()
    params[parsed_url['key']] = parsed_url['value']
    
    return {
        'section': match.group(1),
        'semester': match.group(4),
        'year': {
            'start': match.group(2),
            'end': match.group(3)
        },
        'params': params
    }



In [180]:
all_links = form_result_soup('a', class_='ww_x_GPS')

In [181]:
links = [link for link in all_links if link.text.strip() != 'Tous' ]

In [182]:
links

[<a class="ww_x_GPS" href="javascript:void(0)" onclick="loadReport('ww_x_GPS=2021043255');return false;">Informatique, 2016-2017, Bachelor semestre 1</a>]

In [184]:
items = [parse_item(link.text.strip(), link.attrs['onclick'], payload) for link in links]

loadReport('ww_x_GPS=2021043255');return false;
<_sre.SRE_Match object; span=(11, 32), match="'ww_x_GPS=2021043255'">


In [185]:
items

[{'params': {'dummy': 'ok',
   'ww_b_list': '1',
   'ww_c_langue': '',
   'ww_i_reportModelXsl': '133685270',
   'ww_i_reportmodel': '133685247',
   'ww_x_GPS': '2021043255',
   'ww_x_HIVERETE': 'null',
   'ww_x_PERIODE_ACAD': '355925344',
   'ww_x_PERIODE_PEDAGO': '249108',
   'ww_x_UNITE_ACAD': '249847',
   'zz_x_HIVERETE': '',
   'zz_x_PERIODE_ACAD': '2016-2017',
   'zz_x_PERIODE_PEDAGO': 'Bachelor semestre 1',
   'zz_x_UNITE_ACAD': 'Informatique'},
  'section': 'Informatique',
  'semester': 'Bachelor semestre 1',
  'year': {'end': '2017', 'start': '2016'}}]

In [192]:
list_page_url = isa_url('!GEDPUBLICREPORTS.html')

def load_page(item):
    return requests.get(list_page_url, item['params'])

In [193]:
page = load_page(items[0])

In [196]:
page.text

'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><link rel="stylesheet" type="text/css" href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><fieldset style="text-align:right; width:40%; position:relative; margin-right: 10px;float:right; border: 0; padding: 0 0 8px 0;"><a style="color:#990033;" href="!GEDREPORTS.html?ww_x_HIVERETE=null&amp;zz_x_PERIODE_ACAD=2016-2017&amp;zz_x_UNITE_ACAD=Informatique&amp;ww_i_reportmodel=133685247&amp;zz_x_HIVERETE=&amp;dummy=ok&amp;ww_x_GPS=2021043255&amp;ww_c_langue=&amp;ww_x_PERIODE_ACAD=355925344&amp;ww_x_PERIODE_PEDAGO=249108&amp;ww_i_reportModelXsl=133685270&amp;ww_x_UNITE_ACAD=249847&amp;ww_b_list=1&amp;zz_x_PERIODE_PEDAGO=Bachelor+semestre+1">Identification pour accéder aux e-mails<br>Login to access email adresses</a></fieldset><script>\n  function mailList(x) {\n   var vtop = (screen.height-2